# [모듈 8.1] 엔드포인트 배포 스텝 개발

이 노트북에서는 "SageMaker 호스팅 서비스를 사용하여 모델 배포는 다음과 같은 3단계 프로세스" 중에 두번째와 세번째 단계를 진행 합니다.
- HTTPS endpoint에 대한 endpoint configuration 만들기
- HTTPS 엔드포인트 생성

```
## SageMaker 호스팅 서비스를 사용하여 모델 배포는 다음과 같은 3단계 프로세스

### 1. SageMaker 에서 모델 생성 (이 노트북에서 포함하는 내용임)
- 모델을 생성하여 모델 컴포넌트를 찾을 수 있는 위치를 SageMaker 에 알려줍니다.
    - 모델 컴포넌트: 
        - (1) 모델 결과물이 저장되는 S3 경로 
        - (2) 추론 코드를 포함하는 이미지에 대한 도커 레지스트리 경로

### 2. HTTPS endpoint에 대한 endpoint configuration 만들기
- 프로덕션 변형에서 하나 이상의 모델 이름과 SageMaker 에서 각 프로덕션 변형을 호스트하기 위해 시작할 ML 인스턴스를 지정

### 3. HTTPS 엔드포인트 생성
- endpoint configuration을 SageMaker에 제공합니다. SageMaker는 ML 인스턴스를 시작하고 configuration에서 지정된 대로 모델을 배포합니다. endpoint 에서 추론을 받으려면 클라이언트 애플리케이션에서 SageMaker 런타임 HTTPS 엔드포인트로 요청을 전송합니다. API에 대한 자세한 정보는 InvokeEndpoint API를 참조하십시오.
```


## HTTPS endpoint에 대한 endpoint configuration 만들기
- endpoint configuration
    - Amazon SageMaker 호스팅 서비스가 모델을 배포하는 데 사용하는 endpoint configuration 입니다.endpoint configuration에는 주요하게 아래 두가지가 포함이 됩니다. 
        - 한개 이상의 프러뎍션 변형(ProductionVariants) 
            - CreateModel API를 사용하여 생성 된 모델
            - 모델을 로딩할 ML 인스턴스 (예: ml.m4.2xlarge)
            - InitialInstanceCount: ML 인스턴스 갯수
            - InitialVariantWeight: 프덕뎍션 변형에 보낼 트래픽 비율
- 상세 내용은 여기를 참조 하세요. --> [CreateEndpointConfig](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/APIReference/API_CreateEndpointConfig.html)            
            
아래와 같은 코드를 가지고 endpoint configuration을 생성 합니다.
```python
    create_ep_config_response = sagemaker_boto_client.create_endpoint_config(
        EndpointConfigName=endpoint_config_name,
        ProductionVariants=[{
            'InstanceType': instance_type,
            'InitialVariantWeight': 1,
            'InitialInstanceCount': 1,
            'ModelName': model_name,
            'VariantName': 'AllTraffic'
        }]
    )
```
아래는 endpoint configuration 생성한 예시 입니다.

![endpoint-config.png](img/endpoint-config.png)



### HTTPS 엔드포인트 생성
엔드포인트의 생성은 엔드포인트 이름을 명시하고, 생성한 endpoint_config_name 를 기술하면 됩니다.
```python
    create_endpoint_response = sagemaker_boto_client.create_endpoint(
        EndpointName=args.endpoint_name,
        EndpointConfigName=endpoint_config_name)
```

### 프로덕션 환경에서 테스트 모델
- 아래의 그림 예시는 3개의 프더덕트 변형을 만들고, 각각 다름 모델 및 ML 인스턴스를 가지고 있습니다. Endpoint의 트래픽을 70%, 20%, 10% 씩을 처리하게 구성되어 있습니다. 이러한 구성은 특정 모델에 대한 A/B Testing 환경 구성에 주로 사용됩니다.
- 상세한 내용은 여기를 참조하세요. --> [프로덕션 환경에서 테스트 모델](https://docs.aws.amazon.com/sagemaker/latest/dg/model-ab-testing.html)

![model-traffic-distribution.png](img/model-traffic-distribution.png)

### 오토스케일링을 적용한 추론 아키텍쳐
- 아래는 일반적인 오토 스케일링이 적용된 추론 아키텍쳐 입니다.

![sagemaker-endpoints.png](img/sagemaker-endpoints.png)

---
## 엔드 포인트를 개발하기 위해서는 아래 3개의 단계를 수행합니다.

### (1) 로컬 노트북 인스턴스에서 다커 컨테이너로 엔드포인트를 생성
- 로컬 노트북 인스턴스에서 엔드포인트를 생성하기 위해서, Boto3 API (예: CreateModel API()) 대신 Python SDK를 통해서 모델을 생성 (sagemaker.model.Model() 사용)하고, Model.deploy() 를 통해서 엔드포인트를 바로 생성합니다.
- 모델이 잘 동작하는지(에: 인퍼런스 코드의 로직이 잘 수행되는지) 를 확인하는데 목적을 둡니다.
    - 상세 사항은 여기를 참조 하세요.--> [SageMaker Python SDK: Model](https://sagemaker.readthedocs.io/en/stable/api/inference/model.html)


### (2) 로컬에서 Endpoint 생성 스크립트 실행 (Boto3 API 이용)     
- src/deploy_model.py 를 로컬에서 실행하여 세이지 메메이커 호스팅 엔드포인트를 생성합니다.
    - 이 단계의 목적은 src/deploy_model.py 스크립트의 로직확인에 목적을 둡니다.
    
### (3) SageMaker Pipeline 에서 엔드포인트 생성을 수행합니다.
- 상세 사항은 여기에서 확인 하세요. --> [Amazon SageMaker 모델 구축 파이프라인](img/https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/pipelines.html)
    

## 0. 기본 세이지 메이커 정보 및 기본 변수 로딩

In [2]:
import boto3
import sagemaker
import pandas as pd

region = boto3.Session().region_name

role = sagemaker.get_execution_role()

%store -r 

## 1. 환경 셋업

#### 세이지 메이커 호스트 세션 정의

In [2]:

import sagemaker

# 세이지 메이커의 호스트 모드에서 사용할 세션
sagemaker_session = sagemaker.session.Session()


#### 세이지 메이커 로컬 세션 정의

In [ ]:
# 로컬 노트북 인스턴스에서 사용할 로컬 세션
local_session = sagemaker.local.LocalSession()
local_session.config = {'local' : {'local_code':True}}

## 2. 엔드포인트 생성 실행
---

### (1) 로컬 노트북 인스턴스에서 로컬 모드(로컬 다커 컨테이너 사용)로 훈련 코드 실행




### 엔드 포인트에 사용할 변수 정의

- instance_type = 'local' 
    - 로컬 모드를 실행하는 지시자 임.



In [5]:
from datetime import datetime
suffix = datetime.now().microsecond

In [6]:
instance_type='local'
local_endpoint_name = 'local-endpoint-' + str(suffix)
print("local_endpoint_name: ", local_endpoint_name)


local_endpoint_name:  local-endpoint-98218


### 모델 생성

In [7]:
from datetime import datetime
suffix = datetime.now().microsecond
print("suffix: ", suffix)

from sagemaker.model import Model

local_model = Model(
    image_uri=image_uri,
    model_data= train_model_artifact,
    sagemaker_session= local_session,
    role=role
)

suffix:  484471


### 로컬 엔드포인트 생성

- 에러 케이스1
    - 아래와 같이 에러가 발생한 경우에 로컬머신에 다커 컨테이너가 이미 실행 상태일 수 있습니다.
```
Exception in thread Thread-13:
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sagemaker/local/image.py", line 827, in run
    _stream_output(self.process)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sagemaker/local/image.py", line 887, in _stream_output
    raise RuntimeError("Process exited with code: %s" % exit_code)
RuntimeError: Process exited with code: 1
```
- 아래의 그럼 처럼 실행 중인 다커 컨테이너를 확인하고, 중지 시켜주시기 바랍니다.
- ![docke_error_01.png](img/docke_error_01.png)

In [8]:
local_predictor = local_model.deploy(initial_instance_count = 1,
                                     instance_type = instance_type, 
                                     endpoint_name = local_endpoint_name
            )

Attaching to bfe6ufd8f6-algo-1-1szap
bfe6ufd8f6-algo-1-1szap | [2021-06-26:04:47:45:INFO] No GPUs detected (normal if no gpus installed)
bfe6ufd8f6-algo-1-1szap | [2021-06-26:04:47:45:INFO] No GPUs detected (normal if no gpus installed)
bfe6ufd8f6-algo-1-1szap | [2021-06-26:04:47:45:INFO] nginx config: 
bfe6ufd8f6-algo-1-1szap | worker_processes auto;
bfe6ufd8f6-algo-1-1szap | daemon off;
bfe6ufd8f6-algo-1-1szap | pid /tmp/nginx.pid;
bfe6ufd8f6-algo-1-1szap | error_log  /dev/stderr;
bfe6ufd8f6-algo-1-1szap | 
bfe6ufd8f6-algo-1-1szap | worker_rlimit_nofile 4096;
bfe6ufd8f6-algo-1-1szap | 
bfe6ufd8f6-algo-1-1szap | events {
bfe6ufd8f6-algo-1-1szap |   worker_connections 2048;
bfe6ufd8f6-algo-1-1szap | }
bfe6ufd8f6-algo-1-1szap | 
bfe6ufd8f6-algo-1-1szap | http {
bfe6ufd8f6-algo-1-1szap |   include /etc/nginx/mime.types;
bfe6ufd8f6-algo-1-1szap |   default_type application/octet-stream;
bfe6ufd8f6-algo-1-1szap |   access_log /dev/stdout combined;
bfe6ufd8f6-algo-1-1szap | 
bfe6ufd8f6-algo

### 로컬 프리딕터 생성

HTTPS 프로토콜로 엔드포인트에 요청 및 반환시에는 Sequence of Bytes의 형태로 데이터가 전송됩니다. Predictor 같은 중간에 오브젝트를 두면, 입력 데이터의 직렬화 및 결과 데이터의 역직렬화를 합니다. 

- 참고 사항: 
    - [Predictors](https://sagemaker.readthedocs.io/en/stable/api/inference/predictors.html)
    - [SageMaker Python SDK: Deserializers](https://sagemaker.readthedocs.io/en/stable/api/inference/deserializers.html#)

In [9]:
from sagemaker.deserializers import CSVDeserializer
csv_deserializer = CSVDeserializer(accept='text/csv') # 디폴트가 accept가 'text/csv' 이지만 직관적 이유로 기술함.

def get_predictor(endpoint_name, session, csv_deserializer):
    '''
    predictor = get_predictor(endpoint_name, session, csv_deserializer)    
    '''
    predictor = sagemaker.predictor.Predictor(
        endpoint_name=endpoint_name,
        sagemaker_session= session, # 세션 할당: 로컬 세션 혹은 세이지 메이커 호스트 세션
        deserializer = csv_deserializer, # byte stream을 csv 형태로 변환하여 제공        
    )
    return predictor

predictor = get_predictor(local_endpoint_name, local_session, csv_deserializer)


bfe6ufd8f6-algo-1-1szap | 172.18.0.1 - - [26/Jun/2021:04:47:47 +0000] "GET /ping HTTP/1.1" 200 0 "-" "python-urllib3/1.26.4"


### 테스트 세트 데이터 로드 및 추론
- 기존에 전처리된 테스트 데이터를 로딩해서 추론에 사용합니다.

In [10]:
import os

test_preproc_dir_artifact_file = os.path.join(test_preproc_dir_artifact, 'test.csv')

In [11]:
test_df = pd.read_csv(test_preproc_dir_artifact_file)
test_df.head(2)

,fraud,vehicle_claim,total_claim_amount,customer_age,months_as_customer,num_claims_past_year,num_insurers_past_5_years,policy_deductable,policy_annual_premium,customer_zip,...,collision_type_missing,incident_severity_Major,incident_severity_Minor,incident_severity_Totaled,authorities_contacted_Ambulance,authorities_contacted_Fire,authorities_contacted_None,authorities_contacted_Police,police_report_available_No,police_report_available_Yes
0,0,6038.102399,15838.102399,39,64,0,1,750,2750,95660,...,0,0,1,0,0,0,1,0,1,0
1,0,12813.265683,27813.265683,41,131,0,1,750,2700,90020,...,1,1,0,0,0,0,0,1,1,0


### 추론에 사용할 Payload 생성 및 추론
- 위의 테스트 데이터를 통해서 엔드포인트에 전달할 CSV 형태의 String을 생성합니다. (payload).
- payload 를 엔드포인트에 제공하면, 확률값을 0726071447134018 을 리턴합니다. 
- 보통 확률값이 0.5 보다 작으면 0 (Non-Fruad), 0.5 보다 크면 1 (Fruad) 로 변환해서 예측값으로 사용합니다.

```
payload: 
 6038.102399076349,15838.10239907635,39,64,0,1,750,2750,95660,2012,1,0,0,9800,1,9,2,16,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,1,0
Reponse: 
[['0.0726071447134018']]

```

In [12]:
from IPython.display import display as dp
 
def get_payload(sample, label_col = 'fraud', verbose=False):    
    '''
    아래왁 같이 ',' 형태의 문자형을 리턴함.
    0,0,750,2,0,2,0,2,1,16596.0,1,18,0,59,1,0,0,4500.0,0,0,1,1,0,52,2020,3,0,0,0,2,1,0,0,0,0,0,0,10,12096.0,1,3000,1,0,1,0    
    '''

    sample = sample.drop(columns=[label_col]) # 레이블 제거

    payload = sample.to_csv(header=None, index=None).splitlines() # 
    payload = payload[0]

    if verbose:
        #dp(sample)
        # print("payload length: \n", len(payload))    
        print("pay load type: ", type(payload))
        print("payload: \n", payload)
    
    return payload

def predict(predictor, payload):
    '''
    프리딕터에 콤마 분리된 문자형과 ContentType을 'text/csv' 로 제공
    참고:
        CSVDeserializer 를 사용하지 않으면 byte stream 으로 제공되기에, 아래와 같이 디코딩 하여 사용함.
        result = float(result.decode())
    '''

    pay_load = get_payload(test_df, label_col='fraud', verbose=True)
    result = predictor.predict(pay_load, initial_args = {"ContentType": "text/csv"})
    
    return result



In [13]:
sample = test_df.sample(1)
payload = get_payload(test_df, label_col = 'fraud', verbose=False)
pred_prob = predict(predictor, payload)
pred_prob

pay load type:  <class 'str'>
payload: 
 6038.102399076349,15838.10239907635,39,64,0,1,750,2750,95660,2012,1,0,0,9800,1,9,2,16,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,1,0
bfe6ufd8f6-algo-1-1szap | [2021-06-26:04:47:52:INFO] No GPUs detected (normal if no gpus installed)
bfe6ufd8f6-algo-1-1szap | [2021-06-26:04:47:52:INFO] Determined delimiter of CSV input is ','


[['0.0726071447134018']]

bfe6ufd8f6-algo-1-1szap | 172.18.0.1 - - [26/Jun/2021:04:47:52 +0000] "POST /invocations HTTP/1.1" 200 18 "-" "python-urllib3/1.26.4"


### 로컬 엔드 포인트 삭제
- 위에서 생성한 모델, enpoint config, endpoint를 삭제 합니다.

In [14]:
def delete_endpoint(client, endpoint_name ,is_del_model=True):
    '''
    model, EndpointConfig, Endpoint 삭제
    '''
    response = client.describe_endpoint(EndpointName=endpoint_name)
    EndpointConfigName = response['EndpointConfigName']
    
    response = client.describe_endpoint_config(EndpointConfigName=EndpointConfigName)
    model_name = response['ProductionVariants'][0]['ModelName']    

#     print("EndpointConfigName: \n", EndpointConfigName)
#     print("model_name: \n", model_name)    

    if is_del_model: # 모델도 삭제 여부 임.
        client.delete_model(ModelName=model_name)    
        
    client.delete_endpoint(EndpointName=endpoint_name)
    client.delete_endpoint_config(EndpointConfigName=EndpointConfigName)    
    
    print(f'--- Deleted model: {model_name}')
    print(f'--- Deleted endpoint: {endpoint_name}')
    print(f'--- Deleted endpoint_config: {EndpointConfigName}')    


In [15]:
client = sagemaker.local.LocalSagemakerClient() # 로컬 세션 생성
delete_endpoint(client, local_endpoint_name )

Gracefully stopping... (press Ctrl+C again to force)
--- Deleted model: sagemaker-xgboost-2021-06-26-04-47-41-936
--- Deleted endpoint: local-endpoint-98218
--- Deleted endpoint_config: local-endpoint-98218


## (2) 로컬에서 Endpoint 생성 스크립트 실행 (Boto3 API 이용)

- **[중요] 아래는 약 8분 정도이 시간이 소요 됩니다.**

In [4]:
# !pygmentize src/deploy_model.py # 코드를 보기 위해서 주석을 제거하세요.

In [16]:
import sagemaker
region = sagemaker.Session().boto_region_name
print("Using AWS Region: {}".format(region))

endpoint_name = 'fraud2scratch-' + str(suffix)
endpoint_instance_type = 'ml.t2.xlarge'

Using AWS Region: ap-northeast-2


In [17]:
%%sh -s "$sagemaker_model" "$region" "$endpoint_instance_type" "$endpoint_name" 
python src/deploy_model.py \
--model_name $1 \
--region $2 \
--endpoint_instance_type $3 \
--endpoint_name $4



#############################################
args.model_name: pipelines-460lbbvwpsmp-fraudscratchmodel-4ap057a5cw
args.region: ap-northeast-2
args.endpoint_instance_type: ml.t2.xlarge
args.endpoint_name: fraud2scratch-484471
Creating endpoint
Endpoint status is creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: InService


### 추론 테스트

In [18]:
predictor = get_predictor(endpoint_name, sagemaker_session, csv_deserializer)

In [19]:
import numpy as np
sample = test_df.sample(1)
payload = get_payload(test_df, label_col = 'fraud', verbose=False)
pred_prob = predict(predictor, payload)
pred_prob

pay load type:  <class 'str'>
payload: 
 6038.102399076349,15838.10239907635,39,64,0,1,750,2750,95660,2012,1,0,0,9800,1,9,2,16,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,1,0


[['0.0726071447134018']]

### 엔드포인트 삭제
- is_del_model=False  를 통해서 생성한 모델은 삭제 하지 않습니다. 아래 파이프라인을 통한 엔드포인트 생성시에 모델을 사용하기 위함입니다.

In [21]:
import boto3
boto3_client = boto3.client('sagemaker')
delete_endpoint(boto3_client, endpoint_name, is_del_model=False )
# delete_endpoint(client, local_endpoint_name )

--- Deleted model: pipelines-460lbbvwpsmp-fraudscratchmodel-4ap057a5cw
--- Deleted endpoint: fraud2scratch-484471
--- Deleted endpoint_config: pipelines-460lbbvwpsmp-fraudscratchmodel-4ap057a5cw-config


## (3) 모델 빌딩 파이프라인에서  실행 
---



### 모델 빌딩 파이프라인 변수 생성



In [22]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)

processing_instance_count = ParameterInteger(
    name="ProcessingInstanceCount",
    default_value=1
)
processing_instance_type = ParameterString(
    name="ProcessingInstanceType",
    default_value="ml.m5.xlarge"
)

## 배포 스텝 정의
[중요] `pipeline_endpoint_name` 에 '_' 언데바를 넣으면 에러가 납니다. '-' 대시는 가능합니다.

In [23]:

local_deploy_code_path = 'src/deploy_model.py'
s3_deploy_code_path = f"s3://{default_bucket}/{project_prefix}/code"
s3_deploy_code_uri = sagemaker.s3.S3Uploader.upload(
    local_path=local_deploy_code_path, 
    desired_s3_uri=s3_deploy_code_path,
)
print("s3_deploy_code_uri: ", s3_deploy_code_uri)

pipeline_endpoint_name = 'pipeline-endpoint-'  + str(suffix)

s3_deploy_code_uri:  s3://sagemaker-ap-northeast-2-057716757052/fraud2scratch/code/deploy_model.py


In [24]:
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.workflow.steps import ProcessingStep

deploy_model_processor = SKLearnProcessor(
    framework_version='0.23-1',
    role= role,
    instance_type="ml.t3.medium",
    instance_count=1,
    base_job_name='fraud-scratch-deploy-model',
    sagemaker_session=sagemaker_session)


deploy_step = ProcessingStep(
    name='DeployModel',
    processor=deploy_model_processor,
    job_arguments=[
        "--model_name", sagemaker_model, 
        "--region", region,
        "--endpoint_instance_type", endpoint_instance_type,
        "--endpoint_name", pipeline_endpoint_name
    ],
    code=s3_deploy_code_uri)

### 파리마터, 단계, 조건을 조합하여 최종 파이프라인 정의



In [25]:
from sagemaker.workflow.pipeline import Pipeline


pipeline_name = project_prefix
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_type, 
        processing_instance_count,
    ],
    steps=[deploy_step],
)

#### (선택) 파이프라인 정의 확인 

파이프라인을 정의하는 JSON을 생성하고 파이프라인 내에서 사용하는 파라미터와 단계별 속성들이 잘 정의되었는지 확인할 수 있습니다.

In [26]:
import json


definition = json.loads(pipeline.definition())
definition

{'Version': '2020-12-01',
 'Metadata': {},
 'Parameters': [{'Name': 'ProcessingInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.m5.xlarge'},
  {'Name': 'ProcessingInstanceCount', 'Type': 'Integer', 'DefaultValue': 1}],
 'Steps': [{'Name': 'DeployModel',
   'Type': 'Processing',
   'Arguments': {'ProcessingResources': {'ClusterConfig': {'InstanceType': 'ml.t3.medium',
      'InstanceCount': 1,
      'VolumeSizeInGB': 30}},
    'AppSpecification': {'ImageUri': '366743142698.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3',
     'ContainerArguments': ['--model_name',
      'pipelines-460lbbvwpsmp-fraudscratchmodel-4ap057a5cw',
      '--region',
      'ap-northeast-2',
      '--endpoint_instance_type',
      'ml.t2.xlarge',
      '--endpoint_name',
      'pipeline-endpoint-484471'],
     'ContainerEntrypoint': ['python3',
      '/opt/ml/processing/input/code/deploy_model.py']},
    'RoleArn': 'arn:aws:iam::057716757052:role/service-role/AmazonSageMaker-

### 파이프라인을 SageMaker에 제출하고 실행하기 

파이프라인 정의를 파이프라인 서비스에 제출합니다. 함께 전달되는 역할(role)을 이용하여 AWS에서 파이프라인을 생성하고 작업의 각 단계를 실행할 것입니다.   

In [27]:
pipeline.upsert(role_arn=role)

{'PipelineArn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:pipeline/fraud2scratch',
 'ResponseMetadata': {'RequestId': '4e7ee6f1-f3bf-4c2b-860e-0e8025d08a58',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '4e7ee6f1-f3bf-4c2b-860e-0e8025d08a58',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '86',
   'date': 'Sat, 26 Jun 2021 04:54:48 GMT'},
  'RetryAttempts': 0}}

In [28]:
execution = pipeline.start()

### 파이프라인 운영: 파이프라인 대기 및 실행상태 확인

워크플로우의 실행상황을 살펴봅니다. 

In [29]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:pipeline/fraud2scratch',
 'PipelineExecutionArn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:pipeline/fraud2scratch/execution/y9dtb6uqpsj2',
 'PipelineExecutionDisplayName': 'execution-1624683289063',
 'PipelineExecutionStatus': 'Executing',
 'CreationTime': datetime.datetime(2021, 6, 26, 4, 54, 49, 7000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2021, 6, 26, 4, 54, 49, 7000, tzinfo=tzlocal()),
 'CreatedBy': {},
 'LastModifiedBy': {},
 'ResponseMetadata': {'RequestId': '69939b2e-74da-451f-aee7-55aebc3f26db',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '69939b2e-74da-451f-aee7-55aebc3f26db',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '401',
   'date': 'Sat, 26 Jun 2021 04:54:49 GMT'},
  'RetryAttempts': 0}}

In [31]:
execution.wait()

In [32]:
execution.list_steps()

[{'StepName': 'DeployModel',
  'StartTime': datetime.datetime(2021, 6, 26, 4, 54, 49, 450000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2021, 6, 26, 5, 11, 49, 132000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:processing-job/pipelines-y9dtb6uqpsj2-deploymodel-rtm9sxvn99'}}}]

## 9.배포 파이프라인 추론 테스트

In [33]:
import boto3
sagemaker_boto_client = boto3.client('sagemaker')


def is_available_endpoint(endpoint_name, verbose=False):
    '''
    Return True if endpoint is in service, otherise do False
    '''
    response = sagemaker_boto_client.list_endpoints(NameContains=endpoint_name)
    #existing_endpoints = sagemaker_boto_client.list_endpoints(NameContains=endpoint_name)['Endpoints']
    
    if verbose:
        print("Response: \n", response)

            
    EndpointStatus = response['Endpoints'][0]['EndpointStatus']
    if verbose:
        print("EndpointStatus: ", EndpointStatus)
    
        
    if EndpointStatus == 'InService':
        return True
    else:
        return False



if is_available_endpoint(pipeline_endpoint_name, verbose=False):
    pipeline_predictor = get_predictor(pipeline_endpoint_name, sagemaker_session, csv_deserializer)
    sample = test_df.sample(1)
    payload = get_payload(test_df, label_col = 'fraud', verbose=False)
    pred_prob = predict(pipeline_predictor, payload)
    print("pred_prob: \n", pred_prob)
else:
    print("Wait for creating an endpoint")


pay load type:  <class 'str'>
payload: 
 6038.102399076349,15838.10239907635,39,64,0,1,750,2750,95660,2012,1,0,0,9800,1,9,2,16,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,1,0
pred_prob: 
 [['0.0726071447134018']]


### 엔드포인트 삭제

In [34]:
delete_endpoint(boto3_client, pipeline_endpoint_name, is_del_model = True )

--- Deleted model: pipelines-460lbbvwpsmp-fraudscratchmodel-4ap057a5cw
--- Deleted endpoint: pipeline-endpoint-484471
--- Deleted endpoint_config: pipelines-460lbbvwpsmp-fraudscratchmodel-4ap057a5cw-config
